# Predicting Equipment Failure and Scheduling Maintenance 

Problem Statement
Equipment failure is a major cause of downtime in the telecommunications industry, which can
result in significant financial losses and customer dissatisfaction. To minimize downtime and
ensure optimal performance,meso it is crucial to identify potential equipment failures and schedule
maintenance accordingly proactively. This requires the collection and analysis of large amounts
of data generated by various equipment and network sensors.
The deliverable for this project is a data pipeline that can efficiently collect, clean, and analyze
equipment and network sensor data. The pipeline should be designed to identify potential
equipment failures and schedule maintenance proactively, minimizing downtime and improving
overall equipment performance. The data pipeline will be built using Python and PostgreSQL
and with the Postgres database hosted on Google Cloud.

In [4]:
import pandas as pd
import numpy as np
import psycopg2

In [1]:

from google.colab import files
uploaded = files.upload()

Saving equipment_sensor.csv to equipment_sensor.csv
Saving maintenance_records.csv to maintenance_records.csv
Saving network_sensor.csv to network_sensor.csv


In [2]:
pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Extract the data

In [5]:

df1 = pd.read_csv('equipment_sensor.csv')
df1.head()


,ID,date,time,sensor_reading
0,1,2022-03-01,08:00:00,26.7
1,1,2022-03-01,08:15:00,28.4
2,1,2022-03-01,08:30:00,27.8
3,2,2022-03-01,08:00:00,99.1
4,2,2022-03-01,08:15:00,97.5


In [6]:
df2 = pd.read_csv('maintenance_records.csv')
df2.head()

,ID,date,time,equipment_ID,maintenance_type
0,1,2022-03-01,10:00:00,1,Preventive Maintenance
1,2,2022-03-02,14:30:00,2,Corrective Maintenance
2,3,2022-03-03,08:00:00,1,Corrective Maintenance


In [10]:
df3 = pd.read_csv('network_sensor.csv')
df3.head()

,ID,date,time,sensor_reading
0,1,2022-03-01,08:00:00,0.58
1,1,2022-03-01,08:15:00,0.62
2,1,2022-03-01,08:30:00,0.60
3,2,2022-03-01,08:00:00,0.89
4,2,2022-03-01,08:15:00,0.85


Clean the data

In [11]:
transformed_data1= df1.dropna()
transformed_data2= df2.dropna()
transformed_data3= df3.dropna()
transformed_data1



,ID,date,time,sensor_reading
0,1,2022-03-01,08:00:00,26.7
1,1,2022-03-01,08:15:00,28.4
2,1,2022-03-01,08:30:00,27.8
3,2,2022-03-01,08:00:00,99.1
4,2,2022-03-01,08:15:00,97.5
5,2,2022-03-01,08:30:00,98.2


Transform the data
Merge the 3 dataset

In [12]:
df_merged = transformed_data1.merge(transformed_data2,on = "ID").merge(transformed_data3, on="ID" )
df_merged


,ID,date_x,time_x,sensor_reading_x,date_y,time_y,equipment_ID,maintenance_type,date,time,sensor_reading_y
0,1,2022-03-01,08:00:00,26.7,2022-03-01,10:00:00,1,Preventive Maintenance,2022-03-01,08:00:00,0.58
1,1,2022-03-01,08:00:00,26.7,2022-03-01,10:00:00,1,Preventive Maintenance,2022-03-01,08:15:00,0.62
2,1,2022-03-01,08:00:00,26.7,2022-03-01,10:00:00,1,Preventive Maintenance,2022-03-01,08:30:00,0.60
3,1,2022-03-01,08:15:00,28.4,2022-03-01,10:00:00,1,Preventive Maintenance,2022-03-01,08:00:00,0.58
4,1,2022-03-01,08:15:00,28.4,2022-03-01,10:00:00,1,Preventive Maintenance,2022-03-01,08:15:00,0.62
5,1,2022-03-01,08:15:00,28.4,2022-03-01,10:00:00,1,Preventive Maintenance,2022-03-01,08:30:00,0.60
6,1,2022-03-01,08:30:00,27.8,2022-03-01,10:00:00,1,Preventive Maintenance,2022-03-01,08:00:00,0.58
7,1,2022-03-01,08:30:00,27.8,2022-03-01,10:00:00,1,Preventive Maintenance,2022-03-01,08:15:00,0.62
8,1,2022-03-01,08:30:00,27.8,2022-03-01,10:00:00,1,Preventive Maintenance,2022-03-01,08:30:00,0.60
9,2,2022-03-01,08:00:00,99.1,2022-03-02,14:30:00,2,Corrective Maintenance,2022-03-01,08:00:00,0.89


In [14]:
# Load the transformed data into a CSV file
from google.colab import files
df_merged.to_csv('equip_failure.csv', encoding = 'utf-8-sig') 
files.download('equip_failure.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
!curl ipecho.net/plain

35.185.21.140

Data loading

In [ ]:
import psycopg2

conn = psycopg2.connect(
    host="34.16.140.95",
    database="moringa",
    user="postgres",
    password="changeme_123"
)

# create a cursor object
cur = conn.cursor()


# fetch the results
rows = cur.fetchall()

# close the cursor and connection
cur.close()
conn.close()